<a href="https://colab.research.google.com/github/luisosmx/python_exercises/blob/main/03_Primer_scrapeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Setup






In [1]:
! pip install requests
! pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Primer scrapeo

El proceso de scrapear una página web es parecido a lo que hace un humano cuando quiere buscar algo en Internet, la diferencia es que en lugar de ver el contenido presentado por el navegador, el programa analiza y selecciona el código fuente generalmente programado en  HTML y JavaScript.

El primer paso es por tanto seleccionar una página web para hacer el scraping y descargarla. Ya sabemos cómo hacer peticiones HTTP mediante `requests` así que vamos a a hacer una petición de la web de ejemplo por excelencia:

In [1]:
import requests

req = requests.get("https://example.com")

El caso es que como respuesta a la petición se nos ha develto la página y podemos ver su código fuente en crudo:

In [2]:
print(req.text)

<!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1" />
    <style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>    
</head>

<body>
<div>
    <h1>Example Domain</h1>
    <p>This domai

Aquí tenemos un documento HTML bien estructurado con sus etiquetas.

Cuando el navegador interpreta estas etiquetas que se abren y se cierran, con sus atributos y contenidos genera lo que se conoce como **DOM** (*Document Object Model*), una interfaz de programación para documentos HTML y XML que en esencia es como un árbol ramificado de  componentes padres e hijos. El padre de todo es `html`, que tiene dos hijos `head` y `body`, el primero contiene el `title` y los metadatos, el otro el contenido de la página, una capa `div` que a su vez tiene una cabecera `h1` y unos parágrados `p`.

Pues bien, la biblioteca `BeautifulSoup` lo que hace es generar su propia estructura parecida a la interfaz **DOM** pero en Python, creando un árbol con los elementos del documento. Básicamente le pasamos un documento HTML en crudo y ella lo transforma en un objeto dinámico con el que podemos interactuar:

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text)

print(soup)

<!DOCTYPE html>
<html>
<head>
<title>Example Domain</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<style type="text/css">
    body {
        background-color: #f0f0f2;
        margin: 0;
        padding: 0;
        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;
        
    }
    div {
        width: 600px;
        margin: 5em auto;
        padding: 2em;
        background-color: #fdfdff;
        border-radius: 0.5em;
        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);
    }
    a:link, a:visited {
        color: #38488f;
        text-decoration: none;
    }
    @media (max-width: 700px) {
        div {
            margin: 0 auto;
            width: auto;
        }
    }
    </style>
</head>
<body>
<div>
<h1>Example Domain</h1>
<p>This domain is for use in illustrative examples

A simple vista parece lo mismo, pero ahora podemos hacer algo como esto para consultar el título de la página:

In [4]:
soup.select("title")

[<title>Example Domain</title>]

Esto que nos devuelve es un objeto, veamos tu tipo:

In [5]:
type(soup.select("title"))

list

Es un conjunto de resultados que contiene los tags que concuerdan con el nombre `title`, por tanto es una lista.

Veamos qué tipo tiene ese primer valor del conjunto:

In [6]:
type(soup.select("title")[0])

bs4.element.Tag

Como véis es un `Tag` y éste contiene diferentes métodos, como por ejemplo `getText()` para recuperar su contenido:

In [7]:
soup.select("title")[0].getText()

'Example Domain'

Podemos recuperar otros elementos esenciales como la cabecera o los parágrafos:

In [8]:
soup.select("h1")

[<h1>Example Domain</h1>]

In [9]:
soup.select("p")

[<p>This domain is for use in illustrative examples in documents. You may use this
     domain in literature without prior coordination or asking for permission.</p>,
 <p><a href="https://www.iana.org/domains/example">More information...</a></p>]

Fijaros que el segundo parágrafo contiene a su vez un enlace, podemos acceder de forma anidada:

In [10]:
# Seleccionar del segundo parágrafo el primer enlace 
a = soup.select("p")[1].select("a")[0]

# Mostrar su contenido
a.getText()

'More information...'

Las etiquetas tienen valores especiales llamados atributos, como la dirección `href` de un enlace. 

Estos se almacenan como un diccionario del objeto, es muy cómodo acceder a ellos:

In [11]:
# Atributo con la dirección del enlace
a['href']

'https://www.iana.org/domains/example'

Estos valores están mapeados del diccionario `attrs`:

In [12]:
a.attrs.items()

dict_items([('href', 'https://www.iana.org/domains/example')])

Siguiendo esta lógica podemos programar un script que recupere todos los atributos de los metadatos:

In [13]:
for meta in soup.select("meta"):
    for atributo, valor in meta.attrs.items():
        print(f"{atributo}: {valor}")

charset: utf-8
http-equiv: Content-type
content: text/html; charset=utf-8
name: viewport
content: width=device-width, initial-scale=1


Solo con esto os podéis hacer una ideal del potencial que tiene.